# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV Pipe StatsBase"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-01-07 04:16:21 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing Julia package StatsBase...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [2]:
using CSV
using DataFrames
using Pipe
using Statistics
using StatsBase

# Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user).

In [3]:
URL = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user"
download(URL, "user.dsv")

"user.dsv"

# Step 3. Assign it to a variable called users and use the 'user_id' as index

In [4]:
users = CSV.read("user.dsv",DataFrame,delim="|")
users
# Indices don't really exist in Julia DataFrames

,user_id,age,gender,occupation,zip_code
,Int64,Int64,String1,String15,String7
1,1,24,M,technician,85711
2,2,53,F,other,94043
3,3,23,M,writer,32067
4,4,24,M,technician,43537
5,5,33,F,other,15213
6,6,42,M,executive,98101
7,7,57,M,administrator,91344
8,8,36,M,administrator,05201
9,9,29,M,student,01002


# Step 4. See the first 25 entries

In [5]:
first(users, 25)

,user_id,age,gender,occupation,zip_code
,Int64,Int64,String1,String15,String7
1,1,24,M,technician,85711
2,2,53,F,other,94043
3,3,23,M,writer,32067
4,4,24,M,technician,43537
5,5,33,F,other,15213
6,6,42,M,executive,98101
7,7,57,M,administrator,91344
8,8,36,M,administrator,05201
9,9,29,M,student,01002


# Step 5. See the last 10 entries

In [6]:
last(users, 10)

,user_id,age,gender,occupation,zip_code
,Int64,Int64,String1,String15,String7
1,934,61,M,engineer,22902
2,935,42,M,doctor,66221
3,936,24,M,other,32789
4,937,48,M,educator,98072
5,938,38,F,technician,55038
6,939,26,F,student,33319
7,940,32,M,administrator,02215
8,941,20,M,student,97229
9,942,48,F,librarian,78209


# Step 6. What is the number of observations in the dataset?

In [7]:
nrow(users)

943

# Step 7. What is the number of columns in the dataset?

In [8]:
size(users)[2]

5

# Step 8. Print the name of all the columns.

In [9]:
names(users)

5-element Vector{String}:
 "user_id"
 "age"
 "gender"
 "occupation"
 "zip_code"

# Step 10. What is the data type of each column?


In [10]:
eltype.(eachcol(users))

5-element Vector{DataType}:
 Int64
 Int64
 String1
 String15
 String7

# Step 11. Print only the occupation column

In [11]:
users[!,:occupation]

943-element PooledArrays.PooledVector{String15, UInt32, SentinelArrays.ChainedVector{UInt32, Vector{UInt32}}}:
 "technician"
 "other"
 "writer"
 "technician"
 "other"
 "executive"
 "administrator"
 "administrator"
 "student"
 "lawyer"
 "other"
 "other"
 "educator"
 ⋮
 "educator"
 "student"
 "engineer"
 "doctor"
 "other"
 "educator"
 "technician"
 "student"
 "administrator"
 "student"
 "librarian"
 "student"

# Step 12. How many different occupations are in this dataset?

In [20]:
size(unique(users[!,:occupation]))

(21,)

# Step 13. What is the most frequent occupation?

In [27]:
value_counts(dataframe, column) = sort!(collect(countmap(dataframe[!, column])), by = x -> x[2], rev = true)

value_counts (generic function with 1 method)

In [28]:
value_counts(users,"occupation")[1]

"student" => 196

# Step 14. Summarize the DataFrame.


In [23]:
describe(users)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,user_id,472.0,1,472.0,943,0,Int64
2,age,34.052,7,31.0,73,0,Int64
3,gender,,F,,M,0,String1
4,occupation,,administrator,,writer,0,String15
5,zip_code,,00000,,Y1A6B,0,String7


# Step 16. Summarize only the occupation column


In [24]:
describe(users.occupation)

Summary Stats:
Length:         943
Type:           String15
Number Unique:  21


# Step 17. What is the mean age of users?


In [30]:
mean(users[!,:age])

34.05196182396607

# Step 18. What is the age with least occurrence?


In [32]:
value_counts(users,"age")[end]

10 => 1